(A) Creating the "Breakout" environment

In [14]:
from tf_agents.environments import suite_gym
env = suite_gym.load("Breakout-v4")
env

(B) Getting familiar with the environment

In [3]:
env.gym

<OrderEnforcing<PassiveEnvChecker<AtariEnv<Breakout-v4>>>>

In [7]:
env.gym.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT']

In [9]:
print(env.current_time_step())

None


In [10]:
env.observation_spec()

BoundedArraySpec(shape=(210, 160, 3), dtype=dtype('uint8'), name='observation', minimum=0, maximum=255)

In [12]:
env.action_spec()

BoundedArraySpec(shape=(), dtype=dtype('int64'), name='action', minimum=0, maximum=3)

In [13]:
env.time_step_spec()

TimeStep(
{'discount': BoundedArraySpec(shape=(), dtype=dtype('float32'), name='discount', minimum=0.0, maximum=1.0),
 'observation': BoundedArraySpec(shape=(210, 160, 3), dtype=dtype('uint8'), name='observation', minimum=0, maximum=255),
 'reward': ArraySpec(shape=(), dtype=dtype('float32'), name='reward'),
 'step_type': ArraySpec(shape=(), dtype=dtype('int32'), name='step_type')})

In [11]:
#env.step(1)
#env.reset()